## **Single-Runway Constraint Programming Model**

<h4 style="color:#0033a0">Parameters</h4>




\begin{aligned}
p & \quad \text{Number of aircraft} \\
E_i & \quad \text{Earliest landing time for aircraft } i \\
T_i & \quad \text{Target (preferred) landing time for aircraft } i \\
L_i & \quad \text{Latest landing time for aircraft } i \\
e_i & \quad \text{Penalty cost per unit time if } t_i < T_i \ ( \text{earliness cost}) \\
l_i & \quad \text{Penalty cost per unit time if } t_i > T_i \ ( \text{lateness cost}) \\
S_{i,j} & \quad \text{Separation time if aircraft } i \text{ lands immediately before } j \\
\end{aligned}

<br/> <h4 style="color:#0033a0">Decision Variables</h4>

\begin{aligned}
\mathit{pos}_i 
  &\quad \text{Position of aircraft } i \text{ in the landing sequence} 
   & (\mathit{pos}_i \in \{0, \dots, p-1\}) 
\\
t_i 
  &\quad \text{Landing time of aircraft } i 
   & (t_i \ge 0)
\\
E'_i 
  &\quad \text{Earliness of aircraft } i
   & (E'_i \ge 0)
\\
L'_i 
  &\quad \text{Lateness of aircraft } i
   & (L'_i \ge 0)
\\
\end{aligned}



<br/> <h4 style="color:#0033a0">Objective Function and Constraints</h4>

\begin{aligned}
&\textbf{Objective:} 
&& \min \quad \sum_{i=1}^{p} \Bigl(e_i\,E'_i + l_i\,L'_i\Bigr) \\[10pt]
&\textbf{Subject to:}
\\[5pt]
& \mathit{pos}_i \neq \mathit{pos}_j 
&& \forall i \neq j 
\quad \text{(all-different positions)} 
\\[5pt]
& E_i \;\le\; t_i \;\le\; L_i 
&& \forall i 
\\[5pt]
& E'_i = \max\{0,\;T_i - t_i\}, 
\quad L'_i = \max\{0,\;t_i - T_i\}
&& \forall i 
\\[5pt]
& \text{If } \mathit{pos}_i < \mathit{pos}_j 
   \text{ then } t_j \;\ge\; t_i + S_{i,j},
&& \forall i \neq j 
\\[3pt]
& \text{If } \mathit{pos}_j < \mathit{pos}_i 
   \text{ then } t_i \;\ge\; t_j + S_{j,i},
&& \forall i \neq j 
\\[5pt]
& \mathit{pos}_i \in \{0,\dots,p-1\}, 
\; t_i,\,E'_i,\,L'_i \ge 0
\quad \forall i 
\end{aligned}



<br/> <h4 style="color:#0033a0">SOLVING</h4>

In [1]:
def read_data_from_file(filename):
    """
    Reads data from a file with the specified format for an air traffic scheduling problem.

    Args:
        filename (str): The path to the data file.

    Returns:
        tuple: A tuple containing the parsed data:
            - num_planes (int): The number of planes.
            - freeze_time (int): The freeze time.
            - planes_data (list): A list of dictionaries, where each dictionary contains
              the data for a plane.
            - separation_times (list of lists): A 2D list of separation times.
    """
    try:
        with open(filename, "r") as f:
            # Read the first line: number of planes and freeze time
            first_line = f.readline().strip().split()
            num_planes = int(first_line[0])
            freeze_time = int(first_line[1])

            planes_data = []
            separation_times = []
            
            for _ in range(num_planes):
                line = f.readline().strip().split()
                appearance_time = int(line[0])
                earliest_landing_time = int(line[1])
                target_landing_time = int(line[2])
                latest_landing_time = int(line[3])
                penalty_early = float(line[4])
                penalty_late = float(line[5])
                planes_data.append(
                    {
                        "appearance_time": appearance_time,
                        "earliest_landing_time": earliest_landing_time,
                        "target_landing_time": target_landing_time,
                        "latest_landing_time": latest_landing_time,
                        "penalty_early": penalty_early,
                        "penalty_late": penalty_late,
                    }
                )
                
                separation_row = []
                while len(separation_row) < num_planes:
                    line = f.readline().strip().split()
                    separation_row.extend([int(x) for x in line])

                separation_times.append(separation_row)

        return num_planes, freeze_time, planes_data, separation_times

    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return None, None, None, None
    except ValueError:
        print(f"Error: Error reading data in file '{filename}'.")
        return None, None, None, None

In [2]:
filename = "data/airland1.txt"

num_planes, freeze_time, planes_data, separation_times = read_data_from_file(filename)

if num_planes is not None:
    print("Number of planes:", num_planes)
    print("Freeze time:", freeze_time)
    print("\nPlane data:")
    for i, plane in enumerate(planes_data):
        print(f"Plane {i+1}: {plane}")
    print("\nSeparation times:")
    for i, row in enumerate(separation_times):
        print(f"After plane {i+1}: {row}")

Number of planes: 10
Freeze time: 10

Plane data:
Plane 1: {'appearance_time': 54, 'earliest_landing_time': 129, 'target_landing_time': 155, 'latest_landing_time': 559, 'penalty_early': 10.0, 'penalty_late': 10.0}
Plane 2: {'appearance_time': 120, 'earliest_landing_time': 195, 'target_landing_time': 258, 'latest_landing_time': 744, 'penalty_early': 10.0, 'penalty_late': 10.0}
Plane 3: {'appearance_time': 14, 'earliest_landing_time': 89, 'target_landing_time': 98, 'latest_landing_time': 510, 'penalty_early': 30.0, 'penalty_late': 30.0}
Plane 4: {'appearance_time': 21, 'earliest_landing_time': 96, 'target_landing_time': 106, 'latest_landing_time': 521, 'penalty_early': 30.0, 'penalty_late': 30.0}
Plane 5: {'appearance_time': 35, 'earliest_landing_time': 110, 'target_landing_time': 123, 'latest_landing_time': 555, 'penalty_early': 30.0, 'penalty_late': 30.0}
Plane 6: {'appearance_time': 45, 'earliest_landing_time': 120, 'target_landing_time': 135, 'latest_landing_time': 576, 'penalty_earl

In [5]:
from ortools.sat.python import cp_model

def create_or_tools_cp_model_with_permutation(num_planes, freeze_time, planes_data, separation_times):
    """
    Creates an OR-Tools CP-SAT model (and variables) for the single-runway 
    aircraft landing problem using a 'permutation' approach to avoid big-M constraints.
    
    Args:
        num_planes (int): Number of planes.
        freeze_time (int): Freeze time (not used here, can be integrated if needed).
        planes_data (list[dict]): For each plane i:
            {
                'appearance_time': int,
                'earliest_landing_time': int,
                'target_landing_time': int,
                'latest_landing_time': int,
                'penalty_early': float,
                'penalty_late': float
            }
        separation_times (list[list]): S[i][j] = min. separation if plane i lands before j.

    Returns:
        model (cp_model.CpModel),
        variables (dict): {
            "position": [IntVar,...],
            "landing_time": [IntVar,...],
            "earliness": [IntVar,...],
            "lateness": [IntVar,...]
        }
    """
    model = cp_model.CpModel()

    # Extract convenience arrays
    E = [p["earliest_landing_time"] for p in planes_data]
    T = [p["target_landing_time"]   for p in planes_data]
    L = [p["latest_landing_time"]   for p in planes_data]
    cost_e = [p["penalty_early"]    for p in planes_data]
    cost_l = [p["penalty_late"]     for p in planes_data]

    # --- 1) Define Variables ---

    # position[i] in [0..num_planes-1], says which "slot" plane i occupies
    position = [
        model.NewIntVar(0, num_planes - 1, f"position_{i}") 
        for i in range(num_planes)
    ]

    # AllDifferent to ensure we have a permutation of planes
    model.AddAllDifferent(position)

    # landing_time[i] in [0..someLargeNumber]
    # must also respect earliest/latest constraints (we'll add those next)
    landing_time = [
        model.NewIntVar(0, 10000000, f"landing_time_{i}") 
        for i in range(num_planes)
    ]

    # earliness / lateness
    earliness = [
        model.NewIntVar(0, 10000000, f"earliness_{i}")
        for i in range(num_planes)
    ]
    lateness = [
        model.NewIntVar(0, 10000000, f"lateness_{i}")
        for i in range(num_planes)
    ]

    # --- 2) Time Window Constraints ---
    for i in range(num_planes):
        # earliest <= landing_time[i] <= latest
        model.Add(landing_time[i] >= E[i])
        model.Add(landing_time[i] <= L[i])

    # --- 3) Earliness / Lateness definitions ---
    for i in range(num_planes):
        # earliness[i] = max(0, T[i] - landing_time[i])
        # lateness[i]  = max(0, landing_time[i] - T[i])
        model.Add(earliness[i] >= T[i] - landing_time[i])
        model.Add(earliness[i] >= 0)
        model.Add(lateness[i]  >= landing_time[i] - T[i])
        model.Add(lateness[i]  >= 0)

    # --- 4) Separation Constraints via Permutation ---

    # We want: if position[i] < position[j] => landing_time[j] >= landing_time[i] + S[i][j]
    # In CP-SAT, we can't directly write "if position[i] < position[j]", 
    # so we'll define a Boolean reification for "position[i] < position[j]".
    # Then we'll OnlyEnforceIf that Boolean is True or False.
    #
    # We'll do this for each unordered pair (i<j).
    for i in range(num_planes):
        for j in range(i+1, num_planes):
            # Boolean var: iBeforeJ = (position[i] < position[j])
            iBeforeJ = model.NewBoolVar(f"iBeforeJ_{i}_{j}")

            # Constrain (position[i] < position[j]) <=> iBeforeJ = 1
            model.Add(position[i] < position[j]).OnlyEnforceIf(iBeforeJ)
            model.Add(position[i] >= position[j]).OnlyEnforceIf(iBeforeJ.Not())

            # If iBeforeJ then t[j] >= t[i] + S[i][j]
            model.Add(landing_time[j] >= landing_time[i] + separation_times[i][j])\
                 .OnlyEnforceIf(iBeforeJ)

            # If jBeforeI then t[i] >= t[j] + S[j][i]
            model.Add(landing_time[i] >= landing_time[j] + separation_times[j][i])\
                 .OnlyEnforceIf(iBeforeJ.Not())

    # --- 5) Objective: minimize sum of earliness + lateness cost ---
    cost_terms = []
    for i in range(num_planes):
        cost_terms.append(cost_e[i] * earliness[i])
        cost_terms.append(cost_l[i] * lateness[i])
    model.Minimize(sum(cost_terms))

    # --- 6) Return the model and variables ---
    variables = {
        "position": position,
        "landing_time": landing_time,
        "earliness": earliness,
        "lateness": lateness,
    }
    return model, variables


def solve_cp_model_with_permutation(num_planes, freeze_time, planes_data, separation_times):
    """Builds and solves the single-runway CP model with a permutation approach."""
    model, vars_ = create_or_tools_cp_model_with_permutation(
        num_planes, freeze_time, planes_data, separation_times
    )

    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
        print("Status:", solver.StatusName(status))
        print("Objective (total earliness + lateness):", solver.ObjectiveValue())
        print()

        position = vars_["position"]
        landing_time = vars_["landing_time"]
        earliness = vars_["earliness"]
        lateness = vars_["lateness"]

        # Print results
        #  - plane i in position solver.Value(position[i])
        #  - landing time
        for i in range(num_planes):
            pos_val = solver.Value(position[i])
            t_val   = solver.Value(landing_time[i])
            e_      = solver.Value(earliness[i])
            L_      = solver.Value(lateness[i])
            T_      = planes_data[i]["target_landing_time"]
            print(f"Plane {i}: position={pos_val}, landing={t_val}, "
                  f"E'={e_}, L'={L_}, target={T_}")

        # Sort planes by their actual position to see the schedule order
        print("\nSchedule order (by position):")
        schedule = sorted(range(num_planes), key=lambda i: solver.Value(position[i]))
        for k in schedule:
            print(f" -> Plane {k} (pos={solver.Value(position[k])}, land={solver.Value(landing_time[k])})")

    else:
        print("No feasible/optimal solution found. Status:", solver.StatusName(status))



if __name__ == "__main__":
    # Suppose these come from your data file
    if num_planes is not None:
        solve_cp_model_with_permutation(num_planes, freeze_time, planes_data, separation_times)


Status: OPTIMAL
Objective (total earliness + lateness): 700.0

Plane 0: position=7, landing=165, E'=0, L'=10, target=155
Plane 1: position=9, landing=258, E'=0, L'=0, target=258
Plane 2: position=0, landing=98, E'=0, L'=0, target=98
Plane 3: position=1, landing=106, E'=0, L'=0, target=106
Plane 4: position=2, landing=118, E'=5, L'=0, target=123
Plane 5: position=3, landing=126, E'=9, L'=0, target=135
Plane 6: position=4, landing=134, E'=4, L'=0, target=138
Plane 7: position=5, landing=142, E'=0, L'=2, target=140
Plane 8: position=6, landing=150, E'=0, L'=0, target=150
Plane 9: position=8, landing=180, E'=0, L'=0, target=180

Schedule order (by position):
 -> Plane 2 (pos=0, land=98)
 -> Plane 3 (pos=1, land=106)
 -> Plane 4 (pos=2, land=118)
 -> Plane 5 (pos=3, land=126)
 -> Plane 6 (pos=4, land=134)
 -> Plane 7 (pos=5, land=142)
 -> Plane 8 (pos=6, land=150)
 -> Plane 0 (pos=7, land=165)
 -> Plane 9 (pos=8, land=180)
 -> Plane 1 (pos=9, land=258)
